The problem is to predict if a project on donorschoose will not get fully funded within 60 days of posting. 
* This prediction is being done at the time of posting so you can only use data available to you at that time. 
* The data is a file that has one row for each project posted with a column for "date_posted" (the date the project was posted) and a column for "datefullyfunded" (the date the project was fully funded - assumption for this assignment is that all projects were fully funded eventually). 
* The data spans Jan 1, 2012 to Dec 31, 2013. 
* You should have your validation/test set be a rolling window of 6 months (which should give you three test sets). 
* The training sets should be everything from 1/1/12 to the beginning of the test set.

In [1]:
# basic dependencies
import datetime
import numpy as np
import pandas as pd 
import seaborn as sns
import plotnine as p9
import matplotlib.pyplot as plt
from sklearn import preprocessing

# my own library of useful functions
import utils
import exploration as exp
import pipeline as pipe

In [2]:
projraw = utils.read_data('projects_2012_2013', 'csv')

In [3]:
projraw.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Visual Arts,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,4/14/13,5/2/13
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literature & Writing,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,4/7/12,4/18/12
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,Social Sciences,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,1/30/12,4/15/12
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,10/11/12,12/5/12
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literature & Writing,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,1/8/13,3/25/13


In [4]:
utils.find_cols_with_missing(projraw)

school_ncesid               9233
school_metro               15224
school_district              172
primary_focus_subject         15
primary_focus_area            15
secondary_focus_subject    40556
secondary_focus_area       40556
resource_type                 17
grade_level                    3
students_reached              59
dtype: int64

In [5]:
projraw.info()
# results of this make me want to not include secondary focus subject or area as a feature.
# Create boolean features for is_subject... etc. Then I can include nans. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124976 entries, 0 to 124975
Data columns (total 26 columns):
projectid                                 124976 non-null object
teacher_acctid                            124976 non-null object
schoolid                                  124976 non-null object
school_ncesid                             115743 non-null float64
school_latitude                           124976 non-null float64
school_longitude                          124976 non-null float64
school_city                               124976 non-null object
school_state                              124976 non-null object
school_metro                              109752 non-null object
school_district                           124804 non-null object
school_county                             124976 non-null object
school_charter                            124976 non-null object
school_magnet                             124976 non-null object
teacher_prefix                           

Here's what I'm going to do to this dataframe:
* convert date columns to dates
* convert t/f columns to actual booleans
* take the categorical variables and convert them to binary
* what should I normalize??? probably price, students reached

In [6]:
# convert categorical variables to binary
categorical_list = ['school_state',
                    'school_metro',
                    'primary_focus_subject',
                    'primary_focus_area',
                    'secondary_focus_subject',
                    'secondary_focus_area', 
                    'resource_type', 
                    'poverty_level',
                    'grade_level']
proj = utils.make_cat_dummy(projraw, categorical_list)

# create major city dummy for biggest cities
big_cities = ['Los Angeles', 'Chicago', 'Houston', 'Brooklyn', 'Bronx', 'New York']
proj['in_big_city'] = np.where(proj['school_city'].isin(big_cities), 1, 0)

# create dummies for female teacher and teacher with doctorate
proj['teacher_is_female'] = np.where(proj['teacher_prefix'].isin(['Mrs.', 'Ms.']), True, False)
proj['teacher_is_dr'] = np.where(proj['teacher_prefix'] == 'Dr.', True, False)

# replace string true/false with boolean values
proj = utils.convert_to_boolean(proj, ['school_charter',
                                       'school_magnet',
                                      'eligible_double_your_impact_match'], 't', 'f')

# convert date fields and compute whether project is funded within 60 days
proj['date_posted'] = pd.to_datetime(proj.date_posted) 
proj['datefullyfunded'] = pd.to_datetime(proj.datefullyfunded)
proj['funded'] = np.where(proj.datefullyfunded - proj.date_posted <= datetime.timedelta(days=60), 1, 0)

# normalize project price and students reached
proj['total_price_norm'] = preprocessing.scale(proj['total_price_including_optional_support'].astype('float64'))
proj['students_reached_norm'] = preprocessing.scale(proj['students_reached'].astype('float64'))

In [7]:
not_feature_cols = ['teacher_acctid',
                  'schoolid',
                  'school_ncesid',
                  'school_latitude', 
                  'school_longitude',
                  'school_district',
                  'school_county',
                  'teacher_prefix',
                   'date_posted',
                   'datefullyfunded',
                   'funded']
features = list(set(proj.columns).difference(not_feature_cols))
features

['primary_focus_area_Health & Sports',
 'primary_focus_subject_Community Service',
 'primary_focus_subject_Character Education',
 'secondary_focus_subject_History & Geography',
 'secondary_focus_subject_Extracurricular',
 'school_metro_suburban',
 'school_state_IA',
 'school_state_MT',
 'school_state_SC',
 'primary_focus_area_History & Civics',
 'primary_focus_subject_Literature & Writing',
 'school_state_MN',
 'school_state_TN',
 'school_state_GA',
 'school_state_nan',
 'primary_focus_subject_Environmental Science',
 'school_state_OR',
 'primary_focus_subject_Visual Arts',
 'school_state_AK',
 'primary_focus_subject_Applied Sciences',
 'school_state_TX',
 'secondary_focus_subject_Music',
 'school_state_NE',
 'school_state_NC',
 'school_state_RI',
 'school_state_NJ',
 'secondary_focus_subject_Social Sciences',
 'resource_type_Trips',
 'secondary_focus_area_History & Civics',
 'primary_focus_area_Literacy & Language',
 'secondary_focus_subject_Performing Arts',
 'resource_type_Other',
 

In [11]:
X = proj[features]
Y = proj['funded']

from sklearn.model_selection import TimeSeriesSplit
tcv = TimeSeriesSplit(3)

In [106]:
def convert_duration_to_interval(df, date_col, time_interval, time_unit = "weeks", end_with_max = True):
    ''' Takes dataframe, string name of date column, number of intervals
        Will eventually allow different time intervals
        Default option ends last interval with the latest date in the dataframe (so last chunk unequal)

        Returns: list of dates demarcating time intervals
    '''

    df[date_col] = pd.to_datetime(df[date_col])
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    intervals = [min_date]

    if time_unit == "weeks":

        interval_length = datetime.timedelta(weeks = time_interval)
        num_intervals = math.floor((max_date - min_date) / interval_length)

        i = 0
        next_date = min_date
        while i < num_intervals:
            next_date = next_date + interval_length
            intervals.append(next_date)
            i += 1

        # final interval will end with the final date u
        if end_with_max and intervals[-1] < max_date :
            intervals[-1] = max_date

        return intervals
    
    else:
        print("Time unit not yet supported; please convert to weeks")
        return


def create_sliding_window_sets(df, date_col, feature_list, target, time_interval, lag_time):
    ''' Takes full dataframe, string name of date column, list of features, 
        string name of target variable, number of intervals, and any lag time.
        Returns dictionary containing training and testing sets for each interval
    '''

    df[date_col] = pd.to_datetime(df[date_col])
    intervals = convert_duration_to_interval(df, date_col, time_interval)
    
    df['interval'] = pd.cut(df[date_col], intervals)
   
    # we don't want to include any observations too close to train/test date,
    # if we haven't yet observed their outcome
    df['interval'] = np.where(df[date_col] + lag_time > df['interval'].apply(lambda x: x.right), np.nan, df.interval)
    
    return df


In [107]:
intervals = convert_duration_to_interval(proj, 'date_posted', 26)
intervals[0] + datetime.timedelta(days=60)

Timestamp('2012-03-01 00:00:00')

In [108]:
lag_time = datetime.timedelta(days=60)
create_sliding_window_sets(proj, 'date_posted', features, 'funded', 26, lag_time)

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_district,school_county,school_charter,...,grade_level_Grades PreK-2,grade_level_nan,in_big_city,teacher_is_female,teacher_is_dr,funded,total_price_norm,students_reached_norm,bins,interval
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,Pershing Elem Network,Cook,False,...,1,0,1,True,False,1,0.769207,-0.394209,"(2012-12-30, 2013-06-30]","(2012-12-30, 2013-06-30]"
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,Ventura Unif School District,Ventura,False,...,0,0,0,True,False,1,-0.338377,-0.412560,"(2012-01-01, 2012-07-01]","(2012-01-01, 2012-07-01]"
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,Los Angeles Unif Sch Dist,Los Angeles,False,...,0,0,1,True,False,0,0.326379,-0.241286,"(2012-01-01, 2012-07-01]","(2012-01-01, 2012-07-01]"
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,New York City Dept Of Ed,Kings (Brooklyn),False,...,1,0,1,True,False,1,-0.435953,-0.443144,"(2012-07-01, 2012-12-30]","(2012-07-01, 2012-12-30]"
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,Central Islip Union Free SD,Suffolk,False,...,1,0,0,True,False,0,2.674925,0.333703,"(2012-12-30, 2013-06-30]","(2012-12-30, 2013-06-30]"
5,00049ec8ca1f2d08cb13cab31b0b85ec,7149611553c700de9a6099f8a9ce598b,462a5fd93cf9fb5d41eecfd2ea860b19,2.621150e+11,42.740157,-84.525821,Lansing,Lansing School District,Ingham,False,...,0,0,0,True,False,0,-0.162259,-0.492079,"(2012-07-01, 2012-12-30]",NaN
6,0004d2fdbb571237fa53a97e7691440b,926671e209fb977bd5123145c1848ad1,1a994778027ab086dc58ec3b47f74ff0,4.047200e+10,33.059361,-112.037727,Maricopa,Maricopa Unif Sch District 20,Pinal,False,...,0,0,0,True,False,1,-0.239853,-0.357507,"(2012-12-30, 2013-06-30]","(2012-12-30, 2013-06-30]"
7,0004ee26667e751dd51384eb9f30c72e,abe4dabb7864f4c548d230cf9070e03f,8409f70bcd81bc06e4b9efca68eed8f6,6.280501e+10,37.761958,-122.193209,Oakland,Oakland Unified School Dist,Alameda,False,...,0,0,0,False,False,1,2.935478,-0.400326,"(2012-12-30, 2013-06-30]","(2012-12-30, 2013-06-30]"
8,0006a31d45f8d52d217e7c5b55c11f37,3b5fada1ad0e339acc669829071320c4,c6a033f9349ea70659c1891b119680ed,2.307320e+11,44.096641,-70.191734,Lewiston,Lewiston Public Schools,Androscoggin,False,...,0,0,0,True,False,1,0.168248,-0.430910,"(2013-06-30, 2013-12-31]","(2013-06-30, 2013-12-31]"
9,0008ac907bf237a15a959244205d3ee5,92527a5ac5fe946ed1961fb2e1de8cc5,23e34f5d2e2940684269cffe35741598,6.271800e+10,34.381832,-118.531837,Newhall,Newhall School District,Los Angeles,False,...,1,0,0,True,False,1,0.749927,-0.437027,"(2012-07-01, 2012-12-30]","(2012-07-01, 2012-12-30]"


0        2013-06-13
1        2012-06-06
2        2012-03-30
3        2012-12-10
4        2013-03-09
5        2013-01-29
6        2013-05-25
7        2013-04-29
8        2013-10-20
9        2012-12-02
10       2013-09-27
11       2013-08-21
12       2013-03-23
13       2013-12-26
14       2013-07-27
15       2013-02-07
16       2012-03-26
17       2013-03-16
18       2012-03-23
19       2013-02-20
20       2013-02-16
21       2013-11-03
22       2014-02-08
23       2012-06-28
24       2014-01-18
25       2012-05-30
26       2012-07-29
27       2012-03-21
28       2012-10-06
29       2013-11-21
            ...    
124946   2013-02-07
124947   2014-02-04
124948   2012-12-26
124949   2012-12-14
124950   2012-11-24
124951   2013-10-10
124952   2013-08-08
124953   2013-11-13
124954   2012-12-13
124955   2012-11-04
124956   2012-12-13
124957   2012-03-28
124958   2012-12-01
124959   2012-07-29
124960   2012-10-27
124961   2012-10-11
124962   2012-06-02
124963   2013-02-10
124964   2013-06-27
